<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import copy
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Classe Task
Viene definita una classe `Task`, che viene inizializzata a partire da un `DataFrame` e ha come attributi `X`, `y` e le loro suddivisioni per il training e il testing.

La classe contiene anche dei metodi per testare il `Task` con uno specifico classificatore e per scalare l'`X` con uno specifico scaler.

In [93]:
class Task:
  def __init__(self, df):
    self.X = df.drop("Label", axis = 1)
    self.X = self.X.drop("Task", axis = 1)
    self.X = self.X.drop("Id", axis = 1)
    self.y = df["Label"]

    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3, random_state = 0)

  def test(self, clf):
    clf.fit(self.X_train, self.y_train)
    return clf.score(self.X_test, self.y_test)

  def scale(self, sc):
    sc.fit(self.X_train)
    self.X_train = sc.transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

Di seguito vengono letti i `csv` dei Task già ripuliti e vengono salvati come oggetti `Task` all'interno dell'array `tasks`.

In [94]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df)
  tasks.append(task)

# Funzione `test_boosting`
Utilizzando il classificatore passato come argomento, viene eseguito un primo test sui task normali e successivamente un altro test sui task scalati per mezzo dello scaler passato.

Infine viene raccolta la predizione per ogni sample del testing per ogni task, viene calcolata la moda attraverso tutti i task per ogni sample e viene stampata a schermo.

In [95]:
def test_boosting(clf, sc):
  scores = []
  for task in tasks:
    scores.append(task.test(clf))
  scores_mean = np.mean(scores)
  print("score:", scores_mean)

  scaled_tasks = copy.deepcopy(tasks)
  for task in scaled_tasks:
    task.scale(sc)

  scaled_scores = []
  for task in scaled_tasks:
    scaled_scores.append(task.test(clf))
  scaled_scores_mean = np.mean(scaled_scores)
  print("Scaled score:", scaled_scores_mean)

  votes = []
  if scores_mean > scaled_scores_mean:
    for task in scaled_tasks:
      votes.append(clf.predict(task.X_test))
  else:
    for task in tasks:
      votes.append(clf.predict(task.X_test))
  mode, count = stats.mode(votes)
  for i in range(len(mode)):
    print(mode[i], "| count:", count[i])

# Classificatore `AdaBoost`
Un classificatore AdaBoost è un meta-stimatore che inizia inserendo un classificatore nel set di dati originale e quindi adatta copie aggiuntive del classificatore nello stesso set di dati, ma dove i pesi delle istanze classificate in modo errato vengono regolati in modo tale che i classificatori successivi si concentrino maggiormente sui casi difficili.

In [96]:
clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)
sc = MinMaxScaler(feature_range = (0, 1))
test_boosting(clf, sc)

score: 0.7904040404040404
Scaled score: 0.8257575757575758
1 | count: 8
1 | count: 11
1 | count: 8
1 | count: 8
1 | count: 9
1 | count: 9
1 | count: 9
1 | count: 7
1 | count: 6
1 | count: 6
1 | count: 7
1 | count: 10
1 | count: 9
1 | count: 8
1 | count: 7
1 | count: 9
2 | count: 6
1 | count: 8
1 | count: 10
1 | count: 9
1 | count: 6
2 | count: 6
1 | count: 10
1 | count: 9
1 | count: 9
1 | count: 7
2 | count: 7
1 | count: 8
1 | count: 10
1 | count: 10
1 | count: 8
2 | count: 6
1 | count: 9
1 | count: 10
1 | count: 8
1 | count: 9


# Classificatore Gradient Boosting
Questo algoritmo costruisce un modello additivo in modo graduale; consente l'ottimizzazione di funzioni di perdita differenziabili arbitrarie. In ogni fase, `n_classes_` alberi di regressione sono adattati al gradiente negativo della funzione di perdita, ad es. perdita di log binaria o multiclasse.

In [97]:
clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.01, random_state = 0)
sc = MinMaxScaler(feature_range = (0, 1))
test_boosting(clf, sc)

score: 0.8762626262626263
Scaled score: 0.8712121212121212
2 | count: 11
2 | count: 8
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 10
2 | count: 11
2 | count: 10
2 | count: 11
2 | count: 10
2 | count: 10
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 10
2 | count: 7
2 | count: 10
2 | count: 10
2 | count: 11
2 | count: 9
2 | count: 11
2 | count: 11
2 | count: 10
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 9
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 10
2 | count: 11
2 | count: 11
2 | count: 10


# Albero di classificazione Gradient Boosting basato sugli istogrammi
Questo stimatore ha il supporto nativo per i valori mancanti (`NaN`). Durante l'addestramento, il coltivatore di alberi impara ad ogni punto di divisione se i campioni con valori mancanti devono andare al figlio sinistro o destro, in base al potenziale guadagno. Durante la previsione, i campioni con valori mancanti vengono assegnati di conseguenza al figlio sinistro o destro. Se durante l'addestramento non sono stati rilevati valori mancanti per una determinata funzionalità, i campioni con valori mancanti vengono mappati a qualunque figlio abbia il maggior numero di campioni.

In [98]:
clf = HistGradientBoostingClassifier(random_state = 0)
sc = MinMaxScaler(feature_range = (0, 1))
test_boosting(clf, sc)

score: 0.8661616161616162
Scaled score: 0.8661616161616162
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 10
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11
2 | count: 11


# Classificatore Percettrone multi-strato
Vengono scalati i task tramite uno `StandardScaler`, in seguito viene eseguita una ricerca sui parametri forniti, e viene testato su tutti i task un `MLPCLassifier` con i parametri migliori.

Infine viene raccolta la predizione per ogni sample del testing per ogni task, viene calcolata la moda attraverso tutti i task per ogni sample e viene stampata a schermo.

In [ ]:
sc = StandardScaler()
scaled_tasks = copy.deepcopy(tasks)
for task in scaled_tasks:
  task.scale(sc)

parametri_mlp = {
  'hidden_layer_sizes': [(150, 100, 50), (120, 80, 40), (100, 66, 33)],
  'solver': ['lbfgs', 'sgd', 'adam'],
  'alpha': [0.0001, 0.1, 100],
  'max_iter': [100, 200, 300]
}
grid_mlp = HalvingGridSearchCV(MLPClassifier(), parametri_mlp, n_jobs = -1, cv = 3)

scores = []
for task in scaled_tasks:
  grid_mlp.fit(task.X_train, task.y_train)

  clf = MLPClassifier(
    hidden_layer_sizes = grid_mlp.best_params_["hidden_layer_sizes"],
    solver = grid_mlp.best_params_["solver"],
    alpha = grid_mlp.best_params_["alpha"],
    max_iter = grid_mlp.best_params_["max_iter"],
    random_state = 0
  )

  scores.append(task.test(clf))
print("score:", np.mean(scores))

votes = []
for task in scaled_tasks:
  votes.append(clf.predict(task.X_test))
mode, count = stats.mode(votes)
for i in range(len(mode)):
  print(mode[i], "| count:", count[i])

score: 0.7979797979797979
